# Merging Data Gemastik

Data yang akan disatukan didapat dari BPS , berikut adalah data yang dipakai :


| No  | Nama Data                                                                             | Nama Kolom                                                          | Sumber Data                                                                                                                                           |
|-----|---------------------------------------------------------------------------------------|---------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1.  | PDRB (Harga Konstan)                                                                  | PDRB                                                                | https://www.bps.go.id/indicator/52/286/1/-seri-2010-produk-domestik-regional-bruto-.html                                                              |
| 2.  | Persentase penduduk miskin                                                            | persentase kemiskinan                                               | https://www.bps.go.id/indicator/23/192/1/persentase-penduduk-miskin-p0-menurut-provinsi-dan-daerah.html                                               |
| 3.  | Persentase jumlah siswa SMA yang lulus                                                | persentase_lulus_sma                                                | https://www.bps.go.id/indicator/28/1980/1/tingkat-penyelesaian-pendidikan-menurut-jenjang-pendidikan-dan-provinsi.html                                |
| 4.  | Tingkat pengangguran terbuka                                                          | persentase_pengangguran                                             | https://www.bps.go.id/indicator/6/543/2/tingkat-pengangguran-terbuka-menurut-provinsi.html                                                            |
| 5.  | Angka harapan lama sekolah                                                            | harapan_lama_sekolah                                                | https://www.bps.go.id/indicator/26/415/1/-metode-baru-rata-rata-lama-sekolah.html                                                                     |
| 6   | Pengeluaran perkapita                                                                 | pengeluaran_perkapita                                               | https://www.bps.go.id/indicator/26/416/1/-metode-baru-pengeluaran-per-kapita-disesuaikan.html                                                         |
| 7.  | Angka harapan hidup                                                                   | angka_harapan_hidup                                                 | https://www.bps.go.id/indicator/26/414/1/-metode-baru-umur-harapan-hidup-saat-lahir-uhh-.html                                                         |
| 8.  | Proporsi Rumah Tangga Yang Memiliki Akses Terhadap Layanan Sanitasi Layak             | persentase_sanitasi_layak                                           | https://www.bps.go.id/indicator/29/1267/1/proporsi-rumah-tangga-yang-memiliki-akses-terhadap-layanan-sanitasi-layak.html                              |
| 9   | Persentase Rumah Tangga yang Memiliki Akses Terhadap Hunian Yang Layak Dan Terjangkau | persentase_rumah_layak                                              | https://www.bps.go.id/indicator/29/1241/1/persentase-rumah-tangga-yang-memiliki-akses-terhadap-hunian-yang-layak-dan-terjangkau-menurut-provinsi.html |
| 10. | Jumlah Penduduk                                                                       | penduduk_ribu_jiwa                                                  | https://sulut.bps.go.id/indicator/12/958/1/jumlah-penduduk-menurut-provinsi-di-indonesia.html                                                         |
| 11. | Panjang Jalan Menurut Provinsi dan Tingkat Kewenangan Pemerintahan                    | jalan negara, jalan provinsi, jalan kabupaten /kota, total panjang  | https://bps.go.id/indikator/indikator/view_data_pub/0000/api_pub/SENxVDdSbVprRGF2Sk91UTdGcVdTdz09/da_10/1                                             |
| 12. | Persentase Rumah Tangga menurut Provinsi, Tipe Daerah dan Sumber Air Minum Layak      | persentase_air_bersih                                               | https://www.bps.go.id/indicator/29/854/1/persentase-rumah-tangga-menurut-provinsi-tipe-daerah-dan-sumber-air-minum-layak.html                         |

Notes : 
 -  Tingkat pengangguran terbuka dihitung berdasarkan rata - rata pengangguran dari bulan Februari sampai Oktober 
 -  Tingkat persentase kemiskinan dihitung berdasarkan rata - rata persentase dari bulan Maret sampai September 
 - Karena keterbatasan data, panjang jalan di tahun 2015 dihitung dengan ekstrapolasi panjang jalan dari tahun 2016 - 2022

In [1]:
import pandas as pd
import numpy as np

In [2]:
def lower_prov(df):
    df.Provinsi = df.Provinsi.str.lower()
    return df

def make_df(location,name):
    df = lower_prov(pd.read_excel(location))
    df = df[['Provinsi',2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015]]
    df = df.melt(id_vars=['Provinsi'], value_vars=[2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015])
    df = df.rename(columns = {'variable':'tahun','value':name}).sort_values(by = ['Provinsi','tahun']).dropna()
    return df

# Variabel Yang Dipakai

In [3]:
dataset = [
    ['sourceData\\pdrb.xlsx','PDRB'],
    ['sourceData\\sanitasi.xlsx','persentase_sanitasi_layak'],
    ['sourceData\\pengangguran.xlsx','persentase_pengangguran'],
    ['sourceData\\lulus_sma.xlsx','persentase_lulus_sma'],
    ['sourceData\\panjang_jalan.xlsx','panjang_jalan'],
    ['sourceData\\penduduk.xlsx','penduduk_ribu_jiwa'],
    ['sourceData\\harapan_lama_sekolah.xlsx','harapan_lama_sekolah'],
    ['sourceData\\air_minum.xlsx','persentase_air_bersih'],
    ['sourceData\\rumah_layak.xlsx','persentase_rumah_layak'],
    ['sourceData\\angka_harapan_hidup.xlsx','angka_harapan_hidup'],
    ['sourceData\\pengeluaran.xlsx','pengeluaran_perkapita'],
    ['sourceData\\average_persentase penduduk_miskin.xlsx','persentase penduduk_miskin'],
]

df_merged = None

for location,name in dataset:
    df_temp = None
    if name == 'panjang_jalan':
        df_temp = lower_prov(pd.read_excel(location)).sort_values(by = ['Provinsi','tahun'])
        df_temp = df_temp.interpolate(method = 'linear',limit_direction = 'backward')
    else:
        df_temp = make_df(location,name)
        if name == 'PDRB':
            df_merged = df_temp
            continue

    temp = df_temp.iloc[:,2:]
    df_merged[temp.columns] = temp
    
df_merged.tahun = pd.to_datetime(df_merged.tahun,format='%Y')

# Merged Dataset

In [4]:
df_merged

,Provinsi,tahun,PDRB,persentase_sanitasi_layak,persentase_pengangguran,persentase_lulus_sma,jalan_negara,jalan_provinsi,jalan_kabupaten_kota,panjang_total,penduduk_ribu_jiwa,harapan_lama_sekolah,persentase_air_bersih,persentase_rumah_layak,angka_harapan_hidup,pengeluaran_perkapita,persentase penduduk_miskin
238,aceh,2015-01-01,129092.66,54.16,8.830,68.16,2102.0,1782.0,18926.0,22810.0,5002.0,13.73,61.23,88.96,69.50,8729.0,17.095
204,aceh,2016-01-01,136843.82,61.02,7.850,74.46,2102.0,1782.0,18926.0,22810.0,5096.0,13.89,63.31,90.91,69.51,9492.0,16.580
170,aceh,2017-01-01,145806.92,62.92,6.980,70.64,2102.0,1782.0,20135.0,24019.0,5190.0,14.13,64.85,92.01,69.52,10220.0,16.405
136,aceh,2018-01-01,155910.98,67.09,6.440,70.68,2102.0,1782.0,20031.0,23915.0,5243.4,14.27,66.48,93.38,69.64,10357.0,15.825
102,aceh,2019-01-01,164162.98,73.16,5.825,69.96,2102.0,1782.0,20013.0,23897.0,5316.3,14.30,85.81,57.41,69.87,11255.0,15.165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,sumatera utara,2018-01-01,741347.43,74.60,5.580,68.34,2632.0,3049.0,33434.0,39115.0,14476.0,13.14,71.95,94.03,68.61,10652.0,9.080
103,sumatera utara,2019-01-01,799608.95,79.59,5.480,65.21,2632.0,3006.0,34750.0,40388.0,14639.4,13.15,90.22,64.65,68.95,10592.0,8.730
69,sumatera utara,2020-01-01,811188.31,81.08,5.810,70.39,2620.0,3006.0,35284.0,40910.0,14799.4,13.23,89.68,67.44,69.10,10675.0,8.945
35,sumatera utara,2021-01-01,859934.26,82.02,6.170,72.81,2632.0,3006.0,35101.0,40739.0,14936.2,13.27,90.89,69.48,69.23,10790.0,8.750


In [5]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 272 entries, 238 to 1
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Provinsi                    272 non-null    object        
 1   tahun                       272 non-null    datetime64[ns]
 2   PDRB                        272 non-null    float64       
 3   persentase_sanitasi_layak   272 non-null    float64       
 4   persentase_pengangguran     272 non-null    float64       
 5   persentase_lulus_sma        272 non-null    float64       
 6   jalan_negara                272 non-null    float64       
 7   jalan_provinsi              272 non-null    float64       
 8   jalan_kabupaten_kota        272 non-null    float64       
 9   panjang_total               272 non-null    float64       
 10  penduduk_ribu_jiwa          269 non-null    float64       
 11  harapan_lama_sekolah        272 non-null    float64       

In [6]:
df_merged.isna().sum()

Provinsi                      0
tahun                         0
PDRB                          0
persentase_sanitasi_layak     0
persentase_pengangguran       0
persentase_lulus_sma          0
jalan_negara                  0
jalan_provinsi                0
jalan_kabupaten_kota          0
panjang_total                 0
penduduk_ribu_jiwa            3
harapan_lama_sekolah          0
persentase_air_bersih         0
persentase_rumah_layak        0
angka_harapan_hidup           0
pengeluaran_perkapita         8
persentase penduduk_miskin    0
dtype: int64

In [12]:
df_merged.to_csv("merged_data.csv")